# カタカナ15文字を自動識別するモデルの構築
# 課題投稿用Notebook
* これは課題投稿用ファイルです。
* このsubmit_katakana.ipynbファイルを編集し、学習済みモデルなどの必要ファイルを同じフォルダにおき、そのフォルダをzipしたものを投稿してください。

In [1]:
import numpy as np
import glob
import os,sys
import pickle

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、2_notebookのutil.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
except:
    print('ローカル環境での実行です')


import util

ローカル環境での実行です


In [2]:
# classを定義する
### SimpleConvNetクラスの実装
from collections import OrderedDict
from common.layers import Convolution, MaxPooling, ReLU, Affine, SoftmaxWithLoss
from common.gradient import numerical_gradient
from common.optimizer import RMSProp

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 pool_param={'pool_size':2, 'pad':0, 'stride':2},
                 hidden_size=100, output_size=10, weight_init_std=0.01, batch_size=100):
        """
        input_size : tuple, 入力の配列形状(チャンネル数、画像の高さ、画像の幅)
        conv_param : dict, 畳み込みの条件
        pool_param : dict, プーリングの条件
        hidden_size : int, 隠れ層のノード数
        output_size : int, 出力層のノード数
        weight_init_std ： float, 重みWを初期化する際に用いる標準偏差
        """
                
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        
        pool_size = pool_param['pool_size']
        pool_pad = pool_param['pad']
        pool_stride = pool_param['stride']
        
        input_size = input_dim[1]
        conv_output_size = (input_size + 2*filter_pad - filter_size) // filter_stride + 1 # 畳み込み後のサイズ(H,W共通)
        # 4_6_を参考にすると、pool_output_size=out_h、
        pool_output_size = (conv_output_size + 2*pool_pad - pool_size) // pool_stride + 1 # プーリング後のサイズ(H,W共通)
        # Day3 p29を参考にすると、filter_num=FN, pool_output_size=OH=OW
        pool_output_pixel = filter_num * pool_output_size * pool_output_size # プーリング後のピクセル総数
        print("class初期化時の値", filter_num, filter_size, filter_pad, input_size, batch_size, weight_init_std)
        
        # 重みの初期化
        self.params = {}
        std = weight_init_std
        # Day3のp29において、filter_num=FN, input_dim[0]=C, filter_size=FH=FW
        self.params['W1'] = std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size) # W1は畳み込みフィルターの重みになる
        self.params['b1'] = np.zeros(filter_num) #b1は畳み込みフィルターのバイアスになる
        self.params['W2'] = std *  np.random.randn(pool_output_pixel, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = std *  np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        # Day3 p20を参考に、畳み込み層→活性化関数→プーリング層→全結合層→活性化関数→全結合層→ソフトマックス関数
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad']) # W1が畳み込みフィルターの重み, b1が畳み込みフィルターのバイアスになる
        self.layers['ReLU1'] = ReLU()
        self.layers['Pool1'] = MaxPooling(pool_h=pool_size, pool_w=pool_size, stride=pool_stride, pad=pool_pad)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['ReLU2'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """
        損失関数
        x : 入力データ
        t : 教師データ
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        print("SimpleConvNet.accuracyのbatch_size", batch_size)
        # batch_size = self.batch_size
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def gradient(self, x, t):
        """勾配を求める（誤差逆伝播法）
        Parameters
        ----------
        x : 入力データ
        t : 教師データ
        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [2]:
def makedataset():
    """
    データセットをつくる関数です。
    自由に編集してください。
    """
    
    # 次の行は変更しないこと
    test_data= util.loaddata()
    
    # 以下は自由に編集しても構いません
    # 必要な前処理をここに記述してください  
    
    # 正規化
    test_data = test_data / test_data.max()
    test_data = test_data.astype('float32')

    # 配列形式変更
    # 溝口さんより。全結合NNに入力する場合であり、CNNの場合は不要
    # test_data = test_data.reshape(-1, 28*28)

    return test_data


def func_predict(test_data, test_label):
    """
    予測する関数
    data : 画像データ
    return loss, accuracy
    引数とreturn以外は、自由に編集してください    
    """
    
    # 以下を自由に編集してください
    with open("katakana_model.pickle", "rb") as f:
        model = pickle.load(f)
    
    accuracy = model.accuracy(test_data, test_label)
    loss  = model.loss(test_data, test_label)
    
    return loss, accuracy # 編集不可


def main():
    """
    編集しないでください。
    """
    # テスト用データをつくる
    test_data = makedataset()

    # 予測し精度を算出する
    util.accuracy(func_predict, test_data)
    
    return


if __name__=="__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: '../1_data/test_data.npy'